In [1]:
import pandas as pd
import re
from selenium import webdriver as wd
from selenium.webdriver.support.ui import WebDriverWait
from bs4 import BeautifulSoup as bs

### 변환할 데이터 불러오기 

In [57]:
df = pd.read_csv('./geoData/japan_encephalitis.csv', encoding='utf8')

In [60]:
df.tail(2)

,병/의원명,전화번호,주소,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11
6495,효성가정의학과의원,043-873-3760,충청북도 음성군 음성읍 중앙로 119,1층,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6496,휴이비인후과의원,043-237-0603,충청북도 청주시 흥덕구 오송읍 오송생명로 206,(백천빌딩) 4층,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Drop column and Rename

In [21]:
df.drop('Unnamed: 5', axis=1, inplace=True)
df.drop('Unnamed: 6', axis=1, inplace=True)
df.drop('Unnamed: 7', axis=1, inplace=True)
df.drop('Unnamed: 8', axis=1, inplace=True)
df.drop('Unnamed: 11', axis=1, inplace=True)

In [22]:
df.rename(columns={'병/의원명': 'name',' 전화번호': 'number',' 주소': 'address',
                         'Unnamed: 3': 'add1', 'Unnamed: 4': 'add2',
                         'Unnamed: 9': 'lat', 'Unnamed: 10': 'lng'}, inplace=True)

In [23]:
df.head(3)

,name,number,address,add1,add2,lat,lng
0,21세기소아청소년과의원,033-745-0815,강원도 원주시 봉화로 27,(단계동),NaN,NaN,NaN
1,365성모가정의학과의원,033-251-3500,강원도 춘천시 후석로 352,(후평동,국제빌딩) 2층,NaN,NaN
2,Dr.하석우의한사랑의원,033-522-2244,강원도 동해시 효자남길 16,(동회동,월드상가) 1층,NaN,NaN


### Fill NaN to space

In [24]:
df.fillna('', inplace=True)

In [25]:
df

,name,number,address,add1,add2,lat,lng
0,21세기소아청소년과의원,033-745-0815,강원도 원주시 봉화로 27,(단계동),,,
1,365성모가정의학과의원,033-251-3500,강원도 춘천시 후석로 352,(후평동,국제빌딩) 2층,,
2,Dr.하석우의한사랑의원,033-522-2244,강원도 동해시 효자남길 16,(동회동,월드상가) 1층,,
3,강릉아산병원,033-610-3799,강원도 강릉시 사천면 방동길 38,-,,,
4,강원대학교병원,033-258-2000,강원도 춘천시 백령로 156,(효자동),,,
5,강원도 삼척의료원,033-572-1141,강원도 삼척시 오십천로 418,(남양동),,,
6,강원도 영월의료원,033-370-9128,강원도 영월군 영월읍 중앙1로 59,,,,
7,강원도 원주의료원,033-760-4546,강원도 원주시 서원대로 387,(개운동),,,
8,강원도속초의료원,033-630-6000,강원도 속초시 영랑호반길 3,(영랑동),,,
9,강훈소아청소년과의원,033-744-7579,강원도 원주시 입춘로 110,(반곡동,파라다이스프라자 301,,


### Crawling to fill lat, lng

In [26]:
driver = wd.Chrome(executable_path='chromedriver.exe')

In [27]:
def getLatLng(addr):
    driver.get('http://f.goodkiss.co.kr/naver/naverMap.html')
    driver.implicitly_wait(1)
    driver.find_element_by_xpath('/html/body/form/span/input[1]').send_keys(addr)
    driver.find_element_by_xpath('/html/body/form/span/input[2]').click()
    driver.implicitly_wait(1)
    soup = bs(driver.page_source, 'lxml')
    lat = float(soup.findAll('td')[4].text)
    lng = float(soup.findAll('td')[5].text)
    return lat, lng 

In [30]:
for i in range(5): #df
    try:
        lat, lng = getLatLng(df.address[i])
    except IndexError:
        lat, lng = 0, 0
    df.lat[i] = lat
    df.lng[i] = lng

In [35]:
df

,name,number,address,add1,add2,lat,lng
0,21세기소아청소년과의원,033-745-0815,강원도 원주시 봉화로 27,(단계동),,127.928,37.3469
1,365성모가정의학과의원,033-251-3500,강원도 춘천시 후석로 352,(후평동,국제빌딩) 2층,127.749,37.8821
2,Dr.하석우의한사랑의원,033-522-2244,강원도 동해시 효자남길 16,(동회동,월드상가) 1층,129.109,37.4877
3,강릉아산병원,033-610-3799,강원도 강릉시 사천면 방동길 38,-,,128.858,37.8184
4,강원대학교병원,033-258-2000,강원도 춘천시 백령로 156,(효자동),,127.745,37.8752
5,강원도 삼척의료원,033-572-1141,강원도 삼척시 오십천로 418,(남양동),,,
6,강원도 영월의료원,033-370-9128,강원도 영월군 영월읍 중앙1로 59,,,,
7,강원도 원주의료원,033-760-4546,강원도 원주시 서원대로 387,(개운동),,,
8,강원도속초의료원,033-630-6000,강원도 속초시 영랑호반길 3,(영랑동),,,
9,강훈소아청소년과의원,033-744-7579,강원도 원주시 입춘로 110,(반곡동,파라다이스프라자 301,,


In [45]:
# 주소 풀네임 
def address_fullName(df):
    pattern = re.compile('[^a-zA-Z0-9가-힣]|NaN')
    for i in range(len(df)):
        add1 = re.sub(pattern, '', df.add1[i])
        add2 = re.sub(pattern, '', df.add2[i])
        address = df.address[i]
        df.address[i] = ' '.join([address,add1,add2])
    df.drop('add1', axis=1, inplace=True)
    df.drop('add2', axis=1, inplace=True)
    return df

In [46]:
df = address_fullName(df)

In [47]:
df['total'] = df.apply(lambda x: dict(name=x['name'],
                                    number=x['number'],
                                    address=x['address'],
                                    lat=x['lat'],
                                    lng=x['lng']),axis=1)

### MongoDB

In [49]:
from pymongo import MongoClient
client = MongoClient('localhost', 27017)
disease_db = client.get_database('disease_db')

In [50]:
japanEncephalitis_collection = disease_db.get_collection('japanEncephalitis1')
japanEncephalitis_collection.insert_many([
    df.total[i] for i in range(len(df))
])

![image](https://user-images.githubusercontent.com/33097467/44586971-76ad7600-a7ec-11e8-9f11-0618a4f3d963.png)
